In [49]:
import os
import PyPDF2
import pandas as pd 
import re 
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
import nltk

# Set NLTK data path and download to project directory
nltk_data_dir = '/workspaces/genai-finance-nlp-analysis/nltk_data'
os.makedirs(nltk_data_dir, exist_ok=True)
nltk.data.path.append(nltk_data_dir)
nltk.download('punkt', download_dir=nltk_data_dir)
nltk.download('stopwords', download_dir=nltk_data_dir)
nltk.download('wordnet', download_dir=nltk_data_dir)

# Paths (scalable for 10+ reports)
REPORT_DIR = '../data/reports'
reports = {
    'McKinsey_State': 'McKinsey_State.pdf',
    'BCG_Reckoning': 'BCG_Reckoning.pdf',
    'EY_FPA': 'EY_FPA.pdf',
    'McKinsey_Bank': 'McKinsey_Bank.pdf'
}

[nltk_data] Downloading package punkt to /workspaces/genai-finance-
[nltk_data]     nlp-analysis/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /workspaces/genai-
[nltk_data]     finance-nlp-analysis/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /workspaces/genai-finance-
[nltk_data]     nlp-analysis/nltk_data...


In [51]:
import os
print(os.listdir('../data/reports'))

['McKinsey_State.pdf', 'EY_FPA.pdf', 'BCG_Reckoning.pdf', 'KPMG_Insights.pdf', 'McKinsey_Bank.pdf']


In [52]:
import os
print(os.getcwd())

/workspaces/genai-finance-nlp-analysis/notebooks


In [53]:
def extract_text_from_pdf(pdf_path):
    """Extract text from PDF. Why? PDFs have structured text for NLP."""
    try:
        text = ''
        with open(pdf_path, 'rb') as file:
            reader = PyPDF2.PdfReader(file)
            for page in reader.pages:
                extracted = page.extract_text()
                if extracted:
                    text += extracted + '\n'
        return text
    except Exception as e:
        print(f"Error reading {pdf_path}: {e}")
        return ''
    
extracted_texts = {}
for name, path in reports.items():
    full_path = os.path.join(REPORT_DIR, path)
    extracted_texts[name] = extract_text_from_pdf(full_path)
    print(f"Extracted {len(extracted_texts[name])} chars from {name}")


Extracted 51611 chars from McKinsey_State
Extracted 27454 chars from BCG_Reckoning
Extracted 93877 chars from EY_FPA
Extracted 195030 chars from McKinsey_Bank


Clean Text

In [55]:
lemmatizer = WordNetLemmatizer()
stop_words = set(stopwords.words('english'))

def clean_text(text):
    """Clean: lower, remove non-alpha, tokenize, lemmatize, remove stops."""
    text = re.sub(r'[^a-zA-Z\s]', '', text.lower())
    tokens = word_tokenize(text)
    cleaned = [lemmatizer.lemmatize(token) for token in tokens if token not in stop_words and len(token) > 3]
    return ' '.join(cleaned)

cleaned_texts = {name: clean_text(text) for name, text in extracted_texts.items() if text}
df = pd.DataFrame(cleaned_texts.items(), columns=['Firm_Report', 'Text'])
df.to_csv('data/cleaned_texts.csv', index=False)
print(df.head())

LookupError: 
**********************************************************************
  Resource [93mpunkt_tab[0m not found.
  Please use the NLTK Downloader to obtain the resource:

  [31m>>> import nltk
  >>> nltk.download('punkt_tab')
  [0m
  For more information see: https://www.nltk.org/data.html

  Attempted to load [93mtokenizers/punkt_tab/english/[0m

  Searched in:
    - '/home/codespace/nltk_data'
    - '/usr/local/python/3.12.1/nltk_data'
    - '/usr/local/python/3.12.1/share/nltk_data'
    - '/usr/local/python/3.12.1/lib/nltk_data'
    - '/usr/share/nltk_data'
    - '/usr/local/share/nltk_data'
    - '/usr/lib/nltk_data'
    - '/usr/local/lib/nltk_data'
    - '/workspaces/genai-finance-nlp-analysis/nltk_data'
**********************************************************************
